In [ ]:
import pandas as pds

pds.set_option('display.max_rows', None)
data = pds.read_csv(r"C:\Users\aurel\Documents\Aurélien\BStorm\Formations\Python - DM & ML\Python_Clustering\6-Sales2.csv")
data

In [ ]:
data.drop(["Unnamed: 0","Week"], axis = 1, inplace = True)

In [ ]:
data

In [ ]:
data["Qty"] = data["Qty"].astype(int)
data

In [ ]:
data = pds.get_dummies(data)
data

In [ ]:
data.drop("Qty", axis = 1, inplace = True)

In [ ]:
data

In [ ]:
data = data.groupby("Customer").sum()
data

In [ ]:
data.shape

In [ ]:
from sklearn.decomposition import PCA

pca_sales = PCA().fit(data)

In [ ]:
pca_sales.explained_variance_ratio_

In [ ]:
pca_sales.explained_variance_

#### Pour info : on peut retrouver les valeurs propres sur base des variances expliquées (et inversément)
#### Les valeurs propres nous permettent d'accepter ou non les CP's selon leur valeur : 
#### si >1 alors on la conserve sinon on ne la garde pas

In [ ]:
val_propre = (pca_sales.explained_variance_ * 16)/100
val_propre

In [ ]:
nb_components = 16
print((val_propre/nb_components)*100)

#### Fin info

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# variance expliquée

plt.bar(range(1,17), pca_sales.explained_variance_, color = 'red')
plt.xlabel("Nombre de composantes")
plt.ylabel("Variance expliquée")
plt.plot(range(1,17), pca_sales.explained_variance_)
plt.show()

# ratio de la variance expliquée

plt.bar(range(1,17), np.cumsum(pca_sales.explained_variance_ratio_), color = 'green')
plt.xlabel("Nombre de composantes")
plt.ylabel("Cumul du ratio de la variance expliquée")
plt.show()

In [ ]:
# on affiche l'apport de chaque feature dans chaque composante
for i in range(0,3):
    print(pca_sales.components_[i])

In [ ]:
data.columns

In [ ]:
for i in range(0,3):
    plt.barh(data.columns, pca_sales.components_[i])
    plt.xticks(rotation = 90)
    plt.title("CP"+ str(i+1) + " Variance expliquée " + str(round(pca_sales.explained_variance_ratio_[i] * 100, 2)) + "%")
    plt.show()

In [ ]:
pca_data = pca_sales.transform(data)
for i in range(0,100):
    print(pca_data[i])

In [ ]:
plt.scatter(pca_data[:,0], pca_data[:,1])
plt.title("Projection de la CP1 et de la CP2")
plt.xlabel("CP1 - Enfants en bas âge")
plt.ylabel("CP2 - Sportifs")
plt.axhline(linewidth = 1 , color = 'red')
plt.axvline(linewidth = 1 , color = 'red')

In [ ]:
# indique si OUI ou NON le client est caractérisé positivement à la CP1 (enfants en bas âge)
print(pca_data[:,0] > 0)

# indique si OUI ou NON le client est caractérisé positivement à la CP2 (sportifs)
print(pca_data[:,1] > 0)

In [ ]:
data["Enfants"] = pca_data[:,0] > 0

data["Sportifs"] = pca_data[:,1] > 0

In [ ]:
data

In [ ]:
data_kmeans = data.drop(["Enfants", "Sportifs"], axis = 1)

### Kmeans sur le dataset originel
#### Va nous permet de comparer les clusters déduits de la PCA avec ceux du KMeans

In [ ]:
from sklearn.cluster import KMeans

all_inertia = []


for i in range(1,11):
    modelKmeans = KMeans(n_clusters = i, random_state = 0).fit(data_kmeans)
    inertia = modelKmeans.inertia_
    all_inertia.append(inertia)


In [ ]:
all_inertia

In [ ]:
nb_clusters = range(1,11)
plt.plot(nb_clusters, all_inertia)
plt.xlabel("Nombre de clusters")
plt.ylabel("Inertie totale")

#### On constate que l'on obtient le même nombre de clusters => 4 
#### Affichons les labels correspondants aux customers afin de les comparer avec les groupements générés grâce à la PCA

In [ ]:
modelKmeans_4 = KMeans(n_clusters = 4, random_state = 0).fit(data_kmeans)

In [ ]:
modelKmeans_4.labels_

In [ ]:
data["Label KMeans"] = modelKmeans_4.labels_
data

#### Dérivons des 2 colonnes Enfants et Sportifs, une colonne de label propre à la PCA ("Label PCA")

In [ ]:
data.loc[(data["Enfants"] == True) & (data["Sportifs"] == False), 'Label PCA'] = 0

data.loc[(data["Enfants"] == False) & (data["Sportifs"] == False), 'Label PCA'] = 1

data.loc[(data["Enfants"] == False) & (data["Sportifs"] == True), 'Label PCA'] = 2

data.loc[(data["Enfants"] == True) & (data["Sportifs"] == True), 'Label PCA'] = 3

In [ ]:
data["Label PCA"] = data["Label PCA"].astype(int)
data

### Comparons les labels générés à partir de la PCA avec ceux de KMeans

In [ ]:
comparaison = data["Label KMeans"] == data["Label PCA"]
comparaison.sum()

### Pour aller plus loin, effectuons un KMeans avec les données générées à partir de la PCA

#### On va créer un KMeans avec les données transformées des 2 premières composantes (CP) (ou + si on veut)

In [ ]:
pca_data[0][0:2]

In [ ]:
for i in range(0,100):
    print(pca_data[i][0:])

In [ ]:
all_inertia_PCA = []

for i in range(1,11):
    modelKmeansPCA = KMeans(n_clusters = i, random_state = 0).fit(pca_data[:, 0:2])
    inertiaPCA = modelKmeansPCA.inertia_
    all_inertia_PCA.append(inertiaPCA)

In [ ]:
all_inertia_PCA

In [ ]:
nb_clusters = range(1,11)
plt.plot(nb_clusters, all_inertia_PCA)
plt.xlabel("Nombre de clusters")
plt.ylabel("Inertie totale")

### On obtient les mêmes conclusions qu'avec le KMeans sans PCA et la PCA simple

In [ ]:
modelKmeansPCA_4 = KMeans(n_clusters = 4, random_state = 0).fit(pca_data[:, 0:2])

In [ ]:
modelKmeansPCA_4.labels_

In [ ]:
data["Label PCA + KMeans"] = modelKmeansPCA_4.labels_

In [ ]:
modelKmeans_4.labels_

In [ ]:
pds.crosstab(modelKmeansPCA_4.labels_, modelKmeans_4.labels_)